In [46]:
#Step 1: Import all necessary libraries
import pandas as pd
import numpy as np
from prince import FAMD, MCA
import matplotlib.pyplot as plt
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVC
from sklearn.preprocessing import PolynomialFeatures
from sklearn.neighbors import KNeighborsRegressor
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE
from collections import Counter
from sklearn.utils import resample
from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score

In [47]:
%%capture
from tqdm import tqdm_notebook as tqdm
tqdm().pandas()

In [48]:
#Step 2: Read in DataFrame
my_list = list(range(100))
for x in tqdm(my_list):
    df = pd.read_csv('../../NYSDOH_BRFSS_SurveyData_2020.csv', encoding = 'cp1252')

<ipython-input-48-20b7606a6b1f>:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for x in tqdm(my_list):


  0%|          | 0/100 [00:00<?, ?it/s]

C:\Users\Urvi\AppData\Roaming\Python\Python39\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (39,151,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [49]:
# View first 5 rows of the data

df.head(5)

,_STATE,IDATE,IMONTH,IDAY,IYEAR,SEQNO,_PSU,COLGSEX,LANDSEX,RESPSLCT,...,PPS_40,PPS_43,PPS_44,PPS_45,PPS_46,PPS_48,PPS_52,SEXNOCON_OTH,NOVEGFRU_OTH,CHILDREN
0,New York,1152020,1,15,2020,2020002528,2020002528,Not asked or Missing,Not asked or Missing,Male,...,Data do not meet the criteria for statistical ...,Data do not meet the criteria for statistical ...,Data do not meet the criteria for statistical ...,Data do not meet the criteria for statistical ...,Data do not meet the criteria for statistical ...,Data do not meet the criteria for statistical ...,Data do not meet the criteria for statistical ...,NaN,NaN,Not asked or Missing
1,New York,1302020,1,30,2020,2020002529,2020002529,Not asked or Missing,Not asked or Missing,Male,...,Data do not meet the criteria for statistical ...,Data do not meet the criteria for statistical ...,Data do not meet the criteria for statistical ...,Data do not meet the criteria for statistical ...,Data do not meet the criteria for statistical ...,Data do not meet the criteria for statistical ...,Data do not meet the criteria for statistical ...,NaN,NaN,Not asked or Missing
2,New York,1152020,1,15,2020,2020002530,2020002530,Not asked or Missing,Female,Not asked or Missing,...,Data do not meet the criteria for statistical ...,Data do not meet the criteria for statistical ...,Data do not meet the criteria for statistical ...,Data do not meet the criteria for statistical ...,Data do not meet the criteria for statistical ...,Data do not meet the criteria for statistical ...,Data do not meet the criteria for statistical ...,NaN,NaN,Not asked or Missing
3,New York,2032020,2,3,2020,2020004509,2020004509,Not asked or Missing,Not asked or Missing,Female,...,Data do not meet the criteria for statistical ...,Data do not meet the criteria for statistical ...,Data do not meet the criteria for statistical ...,Data do not meet the criteria for statistical ...,Data do not meet the criteria for statistical ...,Data do not meet the criteria for statistical ...,Data do not meet the criteria for statistical ...,NaN,NaN,Not asked or Missing
4,New York,2152020,2,15,2020,2020002531,2020002531,Not asked or Missing,Not asked or Missing,Female,...,Data do not meet the criteria for statistical ...,Data do not meet the criteria for statistical ...,Data do not meet the criteria for statistical ...,Data do not meet the criteria for statistical ...,Data do not meet the criteria for statistical ...,Data do not meet the criteria for statistical ...,Data do not meet the criteria for statistical ...,NaN,NaN,Not asked or Missing


In [50]:
#Step 3: Use df_clean for cleaning
df_clean = df.copy()

# Repalce 'Not asked or Missing' and 'Data do not meet the criteria for statistical reliability, 
# data quality or confidentiality (data are suppressed)' with NA
for col in df.columns:
    df_clean[col].replace({'Not asked or Missing' : np.nan}, inplace = True)
    df_clean[col].replace({'Data do not meet the criteria for statistical reliability, data quality or confidentiality (data are suppressed)' : np.nan}, inplace = True)
    
# Drop columns with over 80% missing values
df_clean.dropna(axis = 1, thresh = len(df_clean) * .50, inplace = True)

In [51]:
# Step 4: Replace other values in the target variable like 'Don't know/not sure' and 'Refused' 
df_clean['CVDCRHD4'].replace('Don’t know/Not sure', np.nan, inplace=True)
df_clean['CVDCRHD4'].replace('Refused', np.nan, inplace=True)
df_clean.dropna(subset=['CVDCRHD4'], inplace=True)

In [52]:
#create two different dataframe of majority and minority class 
df_majority = df_clean[(df_clean['CVDCRHD4']=='No')] 
df_minority = df_clean[(df_clean['CVDCRHD4']=='Yes')] 
# upsample minority class
df_minority_upsampled = resample(df_minority, 
                                 replace=True,    # sample with replacement
                                 n_samples= len(df_majority), # to match majority class
                                 random_state=42)  # reproducible results
# Combine majority class with upsampled minority class
df_upsampled = pd.concat([df_minority_upsampled, df_majority])

In [53]:
len(df_upsampled[df_upsampled['CVDCRHD4'] == 'No'])

13950

In [54]:
# Step 5: Encode categorical variables as numeric to calculate correlations
df_clean_categorical = df_upsampled.copy()
cols = list(df_clean_categorical.columns)
for col in cols:
    if str(df_clean_categorical[col].dtype) == 'object':
        df_clean_categorical[col] = df_clean_categorical[col].astype('category').cat.codes

df_clean.head()

,_STATE,IDATE,IMONTH,IDAY,IYEAR,SEQNO,_PSU,CELLSEX,SEXVAR,GENHLTH,...,WTRSOURCE,STRSMEAL,FRUITVEG,MJUSE30,MJNYSMMP,HEPCTEST,HEPCTOLD,_WT2SPLITS,REGION,DSRIPREG
0,New York,1152020,1,15,2020,2020002528,2020002528,NaN,Male,Very good,...,Public Water Supply,Always,NaN,None,No,No,No,5130.843243,NYS exclusive of NYC,Long Island
1,New York,1302020,1,30,2020,2020002529,2020002529,NaN,Male,Very good,...,Public Water Supply,Never,NaN,None,No,No,No,941.550458,NYS exclusive of NYC,Long Island
2,New York,1152020,1,15,2020,2020002530,2020002530,NaN,Female,Good,...,Public Water Supply,Never,NaN,None,No,No,No,3497.366203,NYS exclusive of NYC,Long Island
3,New York,2032020,2,3,2020,2020004509,2020004509,NaN,Female,Good,...,Public Water Supply,Never,Neighborhood,None,No,NaN,NaN,1187.709030,NYS exclusive of NYC,Long Island
4,New York,2152020,2,15,2020,2020002531,2020002531,NaN,Male,Fair,...,Don't Know/Not Sure,Rarely,NaN,None,No,No,No,13364.387863,NYS exclusive of NYC,Long Island


In [55]:
#Step 7: Create correlation matrix to find which features to use for mca
df_clean_corr = df_clean_categorical.corrwith(df_clean_categorical["CVDCRHD4"])
df_clean_corr_abs = df_clean_corr.abs()
df_clean_corr_abs.sort_values(inplace=True, ascending=False)
df_clean_corr_abs

CVDCRHD4    1.000000
_MICHD      0.969759
CVDINFR4    0.516091
_AGE80      0.462383
_AGE_G      0.448140
              ...   
HISPANC3    0.009717
HEIGHT3     0.005210
_MENT14D    0.005200
IDAY        0.002075
_STATE           NaN
Length: 164, dtype: float64

In [56]:
#Only use the top 30 columns, excluding columns like _MICHD, CVDINFR4, which ask about heart attack and coronary disease which is very close to heart disease.
feature_list = list(df_clean_corr_abs[0:50].keys())
feature_list.remove('CVDINFR4')
feature_list.remove('_MICHD')
feature_list

['CVDCRHD4',
 '_AGE80',
 '_AGE_G',
 '_IMPAGE',
 '_AGEG5YR',
 'AGE',
 '_PNEUMO3',
 '_FLSHOT7',
 'SHINGLE2',
 'COLNSCPY',
 'BLDSTOL1',
 'MEDICARE',
 'STOOLDNA',
 'EMPLOY1',
 'FALL12MN',
 'VIRCOLON',
 'SIGMSCPY',
 '_RFHLTH',
 'PNEUVAC4',
 '_DRDXAR2',
 'HAVARTH4',
 '_AGE65YR',
 'DIFFWALK',
 '_ALTETH3',
 'DIABETE4',
 'HLTHCVR1',
 'CHCCOPD2',
 '_EXTETH3',
 '_HCVU651',
 'FALLINJ4',
 'BPHIGH4',
 'CVDSTRK3',
 'VETERAN3',
 '_RFPSA23',
 'SMOKE100',
 '_PHYS14D',
 'CHCKDNY2',
 'BIRTHSEX',
 'FLUSHOT7',
 'CHECKUP1',
 'PHYSHLTH',
 '_STSTR',
 'DIFFDRES',
 'RMVTETH4',
 'DEAF',
 'QSTVER',
 'DIFFALON',
 '_RFPAP35']

In [57]:
df_clean_columns = df_upsampled[feature_list]
df_clean_columns.head()

,CVDCRHD4,_AGE80,_AGE_G,_IMPAGE,_AGEG5YR,AGE,_PNEUMO3,_FLSHOT7,SHINGLE2,COLNSCPY,...,FLUSHOT7,CHECKUP1,PHYSHLTH,_STSTR,DIFFDRES,RMVTETH4,DEAF,QSTVER,DIFFALON,_RFPAP35
1373,Yes,Imputed Age 70 to 74,Age 65 or older,Age 65 or older,Age 70 to 74,Age 65 or older,Yes,Yes,Yes,NaN,...,Yes,Within past year (anytime less than 12 months ...,Number of days,361111,No,None,No,13,No,Missing or Male
7124,Yes,Imputed Age 60 to 64,Age 55 to 64,Age 55 to 64,Age 60 to 64,Age 55 - 64,Age Less Than 65,Age Less Than 65,No,Yes,...,No,Within past year (anytime less than 12 months ...,Number of days,362119,Yes,All,No,21,No,Received a Pap test within the past 3 years.
3968,Yes,Imputed Age 80 or older,Age 65 or older,Age 65 or older,Age 80 or older,Age 65 or older,Yes,Yes,Yes,Yes,...,Yes,Within past 2 years (1 year but less than 2 ye...,None,361092,Yes,"6 or more, but not all",Yes,11,No,Missing or Male
1433,Yes,Imputed Age 80 or older,Age 65 or older,Age 65 or older,Age 80 or older,Age 65 or older,No,No,No,Yes,...,No,Within past year (anytime less than 12 months ...,Number of days,361121,Yes,All,No,13,No,Missing or Male
1075,Yes,Imputed Age 65 to 69,Age 65 or older,Age 65 or older,Age 65 to 69,Age 65 or older,Yes,Yes,Yes,Yes,...,Yes,Within past year (anytime less than 12 months ...,None,361091,No,1 to 5,No,13,No,Missing or Male


In [58]:
len(df_clean_columns[df_clean_columns['CVDCRHD4'] == 'Yes'])

13950

In [59]:
from sklearn.model_selection import train_test_split 

# Drop all missing values
df_cleaned = df_clean_columns.dropna(axis = 0).reset_index(drop = True)
print(df_cleaned['CVDCRHD4'])

# Drop all rows that are Don't know/Not sure or Refused for column we are predicting
df_cleaned = df_cleaned.loc[(df_cleaned['CVDCRHD4'] == 'No') | (df_cleaned['CVDCRHD4'] == 'Yes')]

# Split into X and y
X = df_cleaned.loc[:, df_cleaned.columns != 'CVDCRHD4']
y = df_cleaned['CVDCRHD4']

# Split the data into training and test data set
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.7, test_size = 0.3,random_state = 42)

0        Yes
1        Yes
2        Yes
3        Yes
4        Yes
        ... 
12113     No
12114     No
12115     No
12116     No
12117     No
Name: CVDCRHD4, Length: 12118, dtype: object


In [60]:
'''famd = FAMD(n_components = 70, n_iter = 3, random_state = 42)
X_train_transformed = famd.fit_transform(X_train)
X_train_transformed.head()'''

mca = MCA( n_components=2, n_iter=3, copy=True, check_input=True,engine='auto', random_state=42)
mca = mca.fit(X)
X_train_transformed = mca.fit_transform(X_train)
X_test_transformed = mca.fit_transform(X_test)

In [61]:
#helper function
def get_performance_scores(y_pred, y_true):
    f1 = f1_score(y_true, y_pred, average='macro')
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='macro')
    recall = recall_score(y_true, y_pred, average='macro')
    return [f1, accuracy, precision, recall]

def print_performance_scores(scores):
    print("Accuracy Score = " + str(scores[1]))
    print("Precision Score = " + str(scores[2]))
    print("Recall Score = " + str(scores[3]))
    print("F1 Score = " + str(scores[0]))

In [62]:
RANDOM_SEED = 694
#Run dummy classifier to find the baseline performance
dummy_clf = DummyClassifier(strategy= 'most_frequent').fit(X_train_transformed,y_train)
y_pred = dummy_clf.predict(X_test_transformed)
print_performance_scores(get_performance_scores(train_preds, y_test))

Accuracy Score = 0.6562156215621562
Precision Score = 0.6261643449233931
Recall Score = 0.6206369846992107
F1 Score = 0.622542790941027


In [63]:
clf_lr = LogisticRegression(random_state = RANDOM_SEED).fit(X_train_transformed, y_train)
train_preds = clf_lr.predict(X_test_transformed)
print_performance_scores(get_performance_scores(train_preds, y_test))

Accuracy Score = 0.6328382838283828
Precision Score = 0.8163181067694001
Recall Score = 0.5007479431563201
F1 Score = 0.388987331399281


In [64]:
#Do grid search for hyperparameter tuning
clf = LogisticRegression(random_state = RANDOM_SEED, solver='liblinear')
grid_values = {'penalty': ['l1', 'l2'],'C':[0.001,.009,0.01,.09,1,5,10,25]}
grid_clf_acc = GridSearchCV(clf, param_grid = grid_values,scoring = 'f1_macro')
my_list = list(range(100))
for x in tqdm(my_list):
    grid_clf_acc.fit(X_train_transformed, y_train)

y_pred_acc = grid_clf_acc.predict(X_test_transformed)

print_performance_scores(get_performance_scores(y_pred_acc, y_test))

<ipython-input-64-b8f27912ffa5>:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for x in tqdm(my_list):


  0%|          | 0/100 [00:00<?, ?it/s]

Accuracy Score = 0.6641914191419142
Precision Score = 0.6345496334265514
Recall Score = 0.6274135318825818
F1 Score = 0.6297474316371596


In [65]:
'''
param_grid = {'C': [0.1,1, 10, 100], 'gamma': [1,0.1,0.01,0.001],'kernel': ['rbf', 'poly', 'sigmoid']}

grid = GridSearchCV(SVC(),param_grid,refit=True,verbose=2)

my_list = list(range(100))
for x in tqdm(my_list):
    grid.fit(X_train_transformed,y_train)

print(grid.best_estimator_)
'''
svm = SVC(C=100, gamma=0.1, kernel='poly').fit(X_train_transformed,y_train)

y_pred_acc = svm.predict(X_test_transformed)

print_performance_scores(get_performance_scores(y_pred_acc, y_test))

Accuracy Score = 0.6322882288228823
Precision Score = 0.31614411441144114
Recall Score = 0.5
F1 Score = 0.38736310025273796


C:\Users\Urvi\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [66]:
random_forest = RandomForestClassifier(n_estimators = 100, random_state = RANDOM_SEED )
random_forest.fit(X_train_transformed, y_train)

y_pred = random_forest.predict(X_test_transformed)

print_performance_scores(get_performance_scores(y_pred, y_test))

Accuracy Score = 0.426017601760176
Precision Score = 0.5826609956444211
Recall Score = 0.533118688721284
F1 Score = 0.38310331469906667


In [67]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

rf_clf = RandomForestClassifier(random_state=RANDOM_SEED).fit(X_train_transformed, y_train)

grid_values = {'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]
 }

rand_search_clf = RandomizedSearchCV(estimator = rf_clf, param_distributions = grid_values, n_iter = 1, cv = 3, verbose=2, random_state=RANDOM_SEED, n_jobs = -1)

my_list = list(range(100))
for x in tqdm(my_list):
    rand_search_clf.fit(X_train_transformed[0:10000], y_train[0:10000])


y_pred_rf = rand_search_clf.predict(X_test_transformed)
   


print_performance_scores(get_performance_scores(y_pred_rf, y_test))

<ipython-input-67-1865c3df3d45>:17: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for x in tqdm(my_list):


  0%|          | 0/100 [00:00<?, ?it/s]

Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits


KeyboardInterrupt: 